----
### 02. Feature Engineering

**Objective:** To engineer new features from the existing ones `based EDA observations and conclusion` and the nature of each feature, in order to enhance models’ ability to capture subtle hidden patterns and trends improving prediction capabilities.

---

Based on the EDA, the following groups of features are candidates for engineering, rationale and engineering methods are also documented:

| Group | Feature Names | Rationale | Suitable Engineering |
|-------|---------------|-----------|-------------------|
| Features with non-overlapping medians | `V4, V9, V10, V11, V12, V14, V16, V17` | These features already show good separation between classes, i.e., they tend to occupy different ranges. Transformations can **enhance this separation** and make the difference between classes clearer. | Squaring or square root transformation |
| Highly concentrated (low variance) features | `V2, V5, V7, V8, V20, V21, V23, V27, V28, Amount` | Low variance features may be ignored by models sensitive to variance (e.g., linear models). Scaling ensures these features are **considered during training**. | MinMaxScaler |
| Multimodal feature | `Time` | Multiple peaks suggest subtle sub-patterns. Engineering can **capture hidden patterns**. | - Feature extraction: hour of day, day segment <br> - Binning into categories (morning/afternoon/evening/night) |
| Heavy-tailed / skewed features | `V1` (skewed to the left) | Outliers could dominate learning, especially for linear models. Transformation reduces the effect of extreme values and **stabilizes the feature**. | - Log-transform <br> - Square root or power transformation |
